In [ ]:
import numpy as np
import random
import networkx as nx
import csv
data={}
with open('train.txt','r') as train:
    sources=set()
    sinks=set()
    UG=nx.Graph() # undirected graph
    for line in train:
        neighbours=[int(i) for i in line.split().split("\t")]
        data[neighbours[0]]=set([neighbours[i+1] for i in range(len(neighbours)-1)])
        edges=[(neighbours[0],neighbours[i+1]) for i in range(len(neighbours)-1)]
        UG.add_edges_from(edges)
        train_sources.add(neighbours[0])
        for i in range(len(neighbours)-1):
            sinks.add(neighbours[i+1])
            
from networkx.algorithms import community
comms = list(community.asyn_fluidc(UG,100))
for node in UG.nodes():
    for i in range(len(comms)):
        if node in comms[i]:
            UG.nodes[node]['community'] = i

In [ ]:
def possampling():
    positive=[]
    for i in range(50000):
        rand=random.randint(0,19999)
        sources=(list(train_sources))[rand]
        begin=data[sources] 
        try:
            sinks=random.choice(list(begin))
            positive.append((sources,sinks))
        except:
            pass
    return positive

In [ ]:
def negsampling():
    negative=[]
    for i in range(50000):
        rand=random.randint(0,19999)
        sources=(list(train_sources))[rand]
        begin = data[sources]
        sinks=random.choice(list(begin))
        if sinks not in begin:
            negative.append((sources, sinks))
    return negative

In [ ]:
psample=possampling()
nsample=negsampling()

In [ ]:
with open('test-public.txt') as test:
    testsample = []
    for line in test:
        node = line.split().split("\t")
        try:
            testsample.append((int(node[1]), int(node[2])))
        except:
            pass

In [ ]:
def posfeature():
    features = []
    for sample in psample:
        feature = []
        try:
            preds = nx.resource_allocation_index(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.jaccard_coefficient(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.adamic_adar_index(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.preferential_attachment(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.cn_soundarajan_hopcroft(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.ra_index_soundarajan_hopcroft(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.within_inter_cluster(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            feature.append(1)             
        except:
            pass
        features.append(feature)
    return features

In [ ]:
def negfeature():
    features = []  
    for sample in nsample:
        feature = []
        try:
            preds = nx.resource_allocation_index(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.jaccard_coefficient(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.adamic_adar_index(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.preferential_attachment(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.cn_soundarajan_hopcroft(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.ra_index_soundarajan_hopcroft(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.within_inter_cluster(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            feature.append(0)  
        except:
            pass
        features.append(feature)
    return features

In [ ]:
def testfeature():
    features = []
    for sample in testsample:
        feature = []
        try:
            preds = nx.resource_allocation_index(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.jaccard_coefficient(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.adamic_adar_index(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.preferential_attachment(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.cn_soundarajan_hopcroft(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.ra_index_soundarajan_hopcroft(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
            preds = nx.within_inter_cluster(UG, [sample])
            for u, v, p in preds:
                feature.append(p)
        except:
            pass
        features.append(feature)
    return features

In [ ]:
pfeature = posfeature()
nfeature = negfeature()
data = pfeature + nfeature
def write_train_to_csv(data):
    with open("train.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["RA","JC","AA","PA","CSH","RSH","WIC","Label"])
        writer.writerows(data)
        
write_train_to_csv(data)

def write_test_to_csv(test_data):
    with open("test.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["RA","JC","AA","PA","CSH","RSH","WIC"])
        writer.writerows(test_data)

test_data=generate_test_features()
write_test_to_csv(test_data)